<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

In this lab, you will implement some of the techniques discussed in the lecture.

Below you are given a solution to the previous scenario. Note that it has two serious drawbacks:
 * The output predictions do not sum up to one (i.e. it does not return a distribution) even though the images always contain exactly one digit.
 * It uses MSE coupled with output sigmoid which can lead to saturation and slow convergence

**Task 1.** Use softmax instead of coordinate-wise sigmoid and use log-loss instead of MSE. Test to see if this improves convergence. Hint: When implementing backprop it might be easier to consider these two function as a single block and not even compute the gradient over the softmax values.

**Task 2.** Implement L2 regularization and add momentum to the SGD algorithm. Play with different amounts of regularization and momentum. See if this improves accuracy/convergence.

**Task 3 (optional).** Implement Adagrad, dropout and some simple data augmentations (e.g. tiny rotations/shifts etc.). Again, test to see how these changes improve accuracy/convergence.

**Task 4.** Try adding extra layers to the network. Again, test how the changes you introduced affect accuracy/convergence. As a start, you can try this architecture: [784,100,30,10]


In [1]:
import random
import numpy as np
from torchvision import datasets, transforms

In [2]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()

    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()

    x_train = x_train.reshape((x_train.shape[0],28*28)) / 255.
    x_test = x_test.reshape((x_test.shape[0],28*28)) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1

    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

100%|██████████| 9912422/9912422 [00:00<00:00, 22363907.26it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 68788014.66it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 25663744.00it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 11353116.07it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [3]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

def softmax(z):
    assert len(z.shape) == 2

    z_max = np.max(z, axis=1)
    z_max = z_max[:, np.newaxis]
    e_z = np.exp(z - z_max)

    div = np.sum(e_z, axis=1)
    div = div[:, np.newaxis]

    return e_z / div

def softmax_prime(z):
    return softmax(z) * (1-softmax(z))

def relu(z):
  return np.max(0, z)

def cross_entropy_loss_deriv(ps, ys):
  return ((ps - ys) / (ps * (1 - ps)))

In [17]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        self.momentum_w = [np.zeros_like(w) for w in self.weights]
        self.momentum_b = [np.zeros_like(b) for b in self.biases]

    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.matmul(w, a)+b)
        return softmax(a) # Usage of softmax instead of sigmoid

    def update_mini_batch(self, mini_batch, eta, lambd, gamma):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate
        # gamma is used for momentum calculation
        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)

        self.momentum_w = [gamma * mw - (eta/len(mini_batch[0])) * nw - lambd * w
                        for w, nw, mw in zip(self.weights, nabla_w, self.momentum_w)]
        self.momentum_b = [gamma * mb - (eta/len(mini_batch[0])) * nb
                       for b, nb, mb in zip(self.biases, nabla_b, self.momentum_b)]

        self.weights = [w + mw
                        for w, mw in zip(self.weights, self.momentum_w)]
        self.biases = [b + mb
                       for b, mb in zip(self.biases, self.momentum_b)]

    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        g = x
        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for b, w in zip(self.biases, self.weights):
            f = np.dot(w, g)+b
            fs.append(f)
            g = sigmoid(f)
            gs.append(g)
        # backward pass <- both steps at once
        dLdg = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for w,g in reversed(list(zip(self.weights,gs[1:]))):
            dLdf = np.multiply(dLdg,np.multiply(g,1-g))
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)

        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])]
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)]

        # We want to up
        return (dLdBs,dLdWs)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)

        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)

    def cost_derivative(self, output_activations, y):
        # Usage of Log-loss loss function derivative
        softmax = np.exp(output_activations)
        softmax = softmax / softmax.sum(axis = 0)
        return softmax - y

    def SGD(self, training_data, epochs, mini_batch_size, eta, lambd=0.0, gamma=0.0, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch((x_mini_batch, y_mini_batch), eta, lambd=lambd, gamma=gamma)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=3.0, lambd=0.000003, gamma = 0.95, test_data=(x_test, y_test))



Epoch: 0, Accuracy: 0.9062
Epoch: 1, Accuracy: 0.9126
Epoch: 2, Accuracy: 0.9176
Epoch: 3, Accuracy: 0.9186
Epoch: 4, Accuracy: 0.9306
Epoch: 5, Accuracy: 0.9338
Epoch: 6, Accuracy: 0.9343
Epoch: 7, Accuracy: 0.935
Epoch: 8, Accuracy: 0.9344
Epoch: 9, Accuracy: 0.94
Epoch: 10, Accuracy: 0.9396
Epoch: 11, Accuracy: 0.9375
Epoch: 12, Accuracy: 0.9391
Epoch: 13, Accuracy: 0.9387
Epoch: 14, Accuracy: 0.9427
Epoch: 15, Accuracy: 0.939
Epoch: 16, Accuracy: 0.943
Epoch: 17, Accuracy: 0.9423
Epoch: 18, Accuracy: 0.9416
Epoch: 19, Accuracy: 0.9385
Epoch: 20, Accuracy: 0.9392
Epoch: 21, Accuracy: 0.9401
Epoch: 22, Accuracy: 0.9487
Epoch: 23, Accuracy: 0.9395
Epoch: 24, Accuracy: 0.9403
Epoch: 25, Accuracy: 0.9442
Epoch: 26, Accuracy: 0.9437
Epoch: 27, Accuracy: 0.9482
Epoch: 28, Accuracy: 0.9428
Epoch: 29, Accuracy: 0.9504
Epoch: 30, Accuracy: 0.9454
Epoch: 31, Accuracy: 0.9454
Epoch: 32, Accuracy: 0.9446
Epoch: 33, Accuracy: 0.9441
Epoch: 34, Accuracy: 0.9464
Epoch: 35, Accuracy: 0.9488
Epoch: 

In [ ]:
# L2-reg:
# C = loss + lambda(W^2) * sum() - suma wag^2
# W = W - dC - lambda(W)
# to jest nakladanie kary dodatkowej na algorytm
# dC = dL * lambda(W)


# Momentum:
# jak mamy gradient dla danej wagi, to chcemy trzymac sume tych gradientow dla tej wagi
# tylko zeby one byly podniesione tam do konkretnej potegi
# tylko to srednia geometryczna, czyli kolejny moment to gradient ktory chcemy + gamma * moment, gamma zwykle duza ~ 0.95


# w adegradzie uwazac zeby do niego nie dodawac tej czesci gradientu ktory wychodzi przy regularyzacji
# maja byc to zwykle gradienty a nie te policzone po regularyzacji